**Connect google drive**

In [2]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Feb 10 14:53:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 Arias.xlsx					'Modelos Word2Vec'
'Colab Notebooks'				'Modelos Word2Vec 5 clases'
 ConvocatoriaSinfonIAUniandes			'Modelo Word2Vec Google'
 darknet					'My Drive'
 Datasets					 MyWorld
'Datos Acueducto'				 Oz
 datosCaso5.txt					 Piscicultura
'Facturas Saona diciembre 2019.gsheet'		 tuits_limpios_0.csv
 frec_palabras.csv				 tuits_limpios.csv
'Modelos 5 Clases (Entrenado con pocos datos)'	 yolov3


**1) Clone the Darknet**



In [4]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14691, done.
remote: Total 14691 (delta 0), reused 0 (delta 0), pack-reused 14691
Receiving objects: 100% (14691/14691), 13.25 MiB | 23.89 MiB/s, done.
Resolving deltas: 100% (10000/10000), done.


**2) Compile Darknet using Nvidia GPU**


In [5]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [8]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [9]:
# Create folder on google drive so that we can save there the weights
# !mkdir 'mydrive/yolov3'

In [10]:
!echo "fish" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /content/drive/MyDrive/yolov3' > data/obj.data
!mkdir data/obj

In [11]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-02-10 14:58:25--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  4.14MB/s    in 39s     

2021-02-10 14:59:04 (3.95 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "/Piscicultura" on Google Drive

In [12]:
!unzip /content/drive/MyDrive/Piscicultura/images.zip -d data/obj

Archive:  /content/drive/MyDrive/Piscicultura/images.zip
  inflating: data/obj/A000001_L_00000002.jpg  
  inflating: data/obj/A000001_L_00000002.txt  
  inflating: data/obj/A000001_L_00000003.jpg  
  inflating: data/obj/A000001_L_00000003.txt  
  inflating: data/obj/A000001_L_00000004.jpg  
  inflating: data/obj/A000001_L_00000004.txt  
  inflating: data/obj/A000001_L_00000005.jpg  
  inflating: data/obj/A000001_L_00000005.txt  
  inflating: data/obj/A000001_L_00000006.jpg  
  inflating: data/obj/A000001_L_00000006.txt  
  inflating: data/obj/A000001_L_00000007.jpg  
  inflating: data/obj/A000001_L_00000007.txt  
  inflating: data/obj/A000001_L_00000008.jpg  
  inflating: data/obj/A000001_L_00000008.txt  
  inflating: data/obj/A000001_L_00000009.jpg  
  inflating: data/obj/A000001_L_00000009.txt  
  inflating: data/obj/A000001_L_00000010.jpg  
  inflating: data/obj/A000001_L_00000010.txt  
  inflating: data/obj/A000001_L_00000011.jpg  
  inflating: data/obj/A000001_L_00000011.txt  
  i

In [13]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.494141 0.322917 0.078906 0.120833

['0', '0.494141', '0.322917', '0.078906', '0.120833']
0 data/obj/A000001_R_00000018.txt
0 0.494141 0.322917 0.078906 0.120833
0 0.016797 0.515278 0.033594 0.086111

['0', '0.016797', '0.515278', '0.033594', '0.086111']
0 data/obj/A000001_R_00000018.txt
0 0.016797 0.515278 0.033594 0.086111
0 0.695312 0.440278 0.137500 0.116667

['0', '0.695312', '0.440278', '0.137500', '0.116667']
0 data/obj/A000001_R_00000018.txt
0 0.695312 0.440278 0.137500 0.116667
0 0.870703 0.323611 0.152344 0.225000

['0', '0.870703', '0.323611', '0.152344', '0.225000']
0 data/obj/A000001_R_00000018.txt
0 0.870703 0.323611 0.152344 0.225000
0 0.841797 0.090278 0.060156 0.088889

['0', '0.841797', '0.090278', '0.060156', '0.088889']
0 data/obj/A000001_R_00000018.txt
0 0.841797 0.090278 0.060156 0.088889
0 0.767969 0.053472 0.096875 0.104167

['0', '0.767969', '0.053472', '0.096875', '0.104167']
0 data/obj/A000001_R_00000018.txt
0 0.767969 0.053472 0.096875 0.104167
0 0.925000

In [14]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/A000001_R_00000029.jpg', 'data/obj/A000001_L_00000041.jpg', 'data/obj/A000001_R_00000017.jpg', 'data/obj/A000001_L_00000035.jpg', 'data/obj/A000001_R_00000011.jpg', 'data/obj/A000001_R_00000037.jpg', 'data/obj/A000001_L_00000025.jpg', 'data/obj/A000001_R_00000004.jpg', 'data/obj/A000001_R_00000009.jpg', 'data/obj/A000001_L_00000023.jpg', 'data/obj/A000001_R_00000014.jpg', 'data/obj/A000001_R_00000022.jpg', 'data/obj/A000001_R_00000041.jpg', 'data/obj/A000001_R_00000024.jpg', 'data/obj/A000001_R_00000021.jpg', 'data/obj/A000001_R_00000013.jpg', 'data/obj/A000001_L_00000022.jpg', 'data/obj/A000001_R_00000044.jpg', 'data/obj/A000001_L_00000007.jpg', 'data/obj/A000001_L_00000009.jpg', 'data/obj/A000001_R_00000015.jpg', 'data/obj/A000001_L_00000020.jpg', 'data/obj/A000001_L_00000034.jpg', 'data/obj/A000001_R_00000006.jpg', 'data/obj/A000001_R_00000028.jpg', 'data/obj/A000001_L_00000015.jpg', 'data/obj/A000001_L_00000027.jpg', 'data/obj/A000001_R_00000033.jpg', 'data/obj/A000001_L

In [15]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_training
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 